In [ ]:
%% Divergencia y Rotacional
% Referncia: Ejercicio 7 (el del Rotacional) del Examen 2 (2017_1)
%
% NOTA: Jorge, este es mi codigo, como lo prorgamaria.
% Primeor el Rotacional y la Divergencia en el nivel 2 del modelo
% Luego, mas abajo, el rotacional y la divergencia en superficie, con sus
% variables correspondientes.

%% Con los vientos en el nivel 2 del modelo.

clear all; close all

%Arch = '/home2/Documentos_Ovel/Clases_Jorge/2017_1/wrfout_d01_2012-08-07_00.nc';
Arch = '/home/jorge/Dropbox/TemSelMod/wrfout_d02_2012-08-08_00_UVW.nc';
Times = ncread(Arch,'Times')'; %Lee los tiempos de simulacion
T_1 = length(Times);
U2 = double(squeeze(ncread(Arch,'U',[1 1 2 1], [Inf Inf 1 Inf], [1 1 1 1])));
V2 = double(squeeze(ncread(Arch,'V',[1 1 2 1], [Inf Inf 1 Inf], [1 1 1 1])));

%Para calcular el rotacional con esquema de diferencias centradas usamos
%las latitudes y longitudes correspondientes a cada componente de la
%velocidad.
XLONG_U = ncread(Arch,'XLONG_U',[1,1,1],[Inf,Inf,1],[1,1,1]);
XLAT_U = ncread(Arch,'XLAT_U',[1,1,1],[Inf,Inf,1],[1,1,1]);
XLONG_V = ncread(Arch,'XLONG_V',[1,1,1],[Inf,Inf,1],[1,1,1]);
XLAT_V = ncread(Arch,'XLAT_V',[1,1,1],[Inf,Inf,1],[1,1,1]);
XLONG = ncread(Arch,'XLONG',[1,1,1],[Inf,Inf,1],[1,1,1]);
XLAT = ncread(Arch,'XLAT',[1,1,1],[Inf,Inf,1],[1,1,1]);

plot(XLONG_U,XLAT_U,'bx')
hold on
plot(XLONG_V,XLAT_V,'ro')
plot(XLONG,XLAT,'k+')

plot(XLONG_U(100,100),XLAT_U(100,100),'bs','MarkerEdgeColor',...
    'b','MarkerFaceColor','b')
hold on
plot(XLONG_V(100,100),XLAT_V(100,100),'rs','MarkerEdgeColor',...
    'r','MarkerFaceColor','r')
plot(XLONG(100,100),XLAT(100,100),'ks','MarkerEdgeColor',...
    'k','MarkerFaceColor','k')


%Ya que el rotacional resultará calculado en el centro,
%para graficar necesitamos XLONG y XLAT:
% El rotacional del viento en el segundo nivel del modelo.
R = 6371000; %radio de la Tierra en metros
[nX, nY, nT] = size(XLONG);
rotU2 = zeros(nX, nY);
divU2 = zeros(nX, nY);
figure
for tt=1:40 %T_1
    for ii = 1:nX-1
        for jj = 2:nY-1
            
            %Definimos a nuestros delta x y delta y por separado
            deltX =  R*(XLONG_U(ii+1,jj)-XLONG_U(ii,jj))*(pi/180)*cosd(XLAT(ii,jj));
            deltY =  R*(XLAT_V(ii,jj+1)-XLAT_V(ii,jj))*(pi/180);
            
            rotU2(ii,jj) = (V2(ii+1,jj,tt)-V2(ii,jj,tt))/deltX...
                - (U2(ii+1,jj,tt)-U2(ii+1,jj-1,tt))/deltY;
            
            divU2(ii,jj) = (U2(ii+1,jj,tt)-U2(ii,jj,tt))/deltX...
                + (V2(ii,jj+1,tt)-V2(ii,jj,tt))/deltY;
        end
    end
    T=Times;
    year=T(1,1:4);
    mes=T(1,6:7);
    dia=T(tt,9:10);
    hora=T(tt,12:13);
    mins=T(tt,15:16);
    clf
    subplot(2,1,1)
    pcolor(XLONG,XLAT,rotU2), shading interp, colormap jet
    caxis([-.0005 0.0005])
    colorbar
    %     axis equal
    xlabel('Longitud ')
    ylabel('Latitud')
    title(['Rotacional del viento el ' dia, '-', mes, '-', year, ' a las ',...
        hora, ':', mins ' GMT, t=' num2str(tt)], 'FontSize', 13)
    subplot(2,1,2)
    pcolor(XLONG,XLAT,divU2), shading interp, colormap jet
    caxis([-.0005 0.0005])
    colorbar
    %     axis equal
    xlabel('Longitud ')
    ylabel('Latitud')
    title(['Divergencia del viento el ' dia, '-', mes, '-', year, ' a las ',...
        hora, ':', mins ' GMT, t=' num2str(tt)], 'FontSize', 13)
    
    pause(0.1)
    %     if tt==27
    %         name=(['rot_enr_0_t' num2str(tt) '_OK' ]);
    %         print('-djpeg', name)
    %     end
end
%%
% Calculo del gradiente de temperatura

T2 = ncread(Arch,'T2',[1,1,1],[Inf,Inf,Inf],[1,1,1]);
[nX, nY, nT] = size(XLONG);

for tt=1:40 %T_1
    clf
MagGradT2 = zeros(nX, nY);
    dTdx = zeros(nX, nY);
    dTdy = zeros(nX, nY);
    for ii = 2:nX-1
        for jj = 2:nY-1
            %Definimos a nuestros delta x y delta y por separado
            deltX =  R*(XLONG(ii+1,jj)-XLONG(ii-1,jj))*(pi/180)*2*cosd(XLAT(ii,jj));
            deltY =  R*(XLAT(ii,jj+1)-XLAT(ii,jj-1))*(pi/180)*2;
            %dt/dx
            dTdx(ii,jj) = (T2(ii+1,jj,tt) - T2(ii-1,jj,tt))/deltX;
            %dt/dy
            dTdy(ii,jj) = (T2(ii,jj+1,tt) - T2(ii,jj-1,tt))/deltY;
        end
    end
    MagGradT2 = sqrt(dTdx.^2 + dTdy.^2);
    pcolor(XLONG,XLAT,MagGradT2), shading interp, colorbar
    caxis([0 .0001])
    hold on
    %quiver(XLONG,XLAT,dtdx,dtdy)
    pause(.1)
    
end




%% Con los vientos de superficie.
% En esta seccion esta el mismo calculo pero con los vientos de superficie.
% La diferencia es que cambia el esquma de las diferencias centradas.

clear all; close all

Arch = '/home2/Documentos_Ovel/Clases_Jorge/2017_1/wrfout_d01_2012-08-07_00.nc';
T = ncread(Arch,'Times')'; %Lee los tiempos de simulacion
T_1 = length(T);
U10 = double(squeeze(ncread(Arch,'U10',[1 1 1], [Inf Inf Inf], [1 1 1])));
V10 = double(squeeze(ncread(Arch,'V10',[1 1 1], [Inf Inf Inf], [1 1 1])));

%Para calcular el rotacional con esquema de diferencias centradas usamos
%las latitudes y longitudes correspondientes a cada componente de la
%velocidad.
XLONG_U = ncread(Arch,'XLONG_U',[1,1,1],[Inf,Inf,1],[1,1,1]);
XLAT_V = ncread(Arch,'XLAT_V',[1,1,1],[Inf,Inf,1],[1,1,1]);
XLONG_V = ncread(Arch,'XLONG_V',[1,1,1],[Inf,Inf,1],[1,1,1]);
XLAT_U = ncread(Arch,'XLAT_U',[1,1,1],[Inf,Inf,1],[1,1,1]);
%Ya que el rotacional resultará calculado en el centro,
%para graficar necesitamos XLONG y XLAT:
XLONG = ncread(Arch,'XLONG',[1,1,1],[Inf,Inf,1],[1,1,1]);
XLAT = ncread(Arch,'XLAT',[1,1,1],[Inf,Inf,1],[1,1,1]);
% El rotacional del viento en el segundo nivel del modelo.
R = 6371000; %radio de la Tierra en metros
[nX, nY, nT] = size(XLONG);
rotSup = zeros(nX, nY);
divSup = zeros(nX, nY);
figure
for tt=1:40 %T_1
    for ii = 2:nX-1
        for jj = 2:nY-1
            
            %Definimos a nuestros delta x y delta y por separado
            
            deltX =  R*(XLONG(ii+1,jj)-XLONG(ii-1,jj))*(pi/180)*cosd(XLAT(ii,jj));
            deltY =  R*(XLAT(ii,jj+1)-XLAT(ii,jj-1))*(pi/180);
            
            rotSup(ii,jj) =  (V10(ii,jj+1,tt)-V10(ii,jj-1,tt))/deltX...
                -(U10(ii+1,jj,tt)-U10(ii-1,jj,tt))/deltY;
            
            divSup(ii,jj) = (U10(ii+1,jj,tt)-U10(ii-1,jj,tt))/deltX...
                + (V10(ii,jj+1,tt)-V10(ii,jj-1,tt))/deltY;
            
        end
    end
    year=T(1,1:4);
    mes=T(1,6:7);
    dia=T(tt,9:10);
    hora=T(tt,12:13);
    mins=T(tt,15:16);
    clf
    subplot(2,1,1)
    pcolor(XLONG,XLAT,rotSup), shading interp, colormap jet
    caxis([-.0005 0.0005])
    colorbar
    %     axis equal
    xlabel('Longitud ')
    ylabel('Latitud')
    title(['Rotacional del viento el ' dia, '-', mes, '-', year, ' a las ', hora, ':', mins ' GMT, t=' num2str(tt)], 'FontSize', 13)
    subplot(2,1,2)
    pcolor(XLONG,XLAT,divSup), shading interp, colormap jet
    caxis([-.0005 0.0005])
    colorbar
    %     axis equal
    xlabel('Longitud ')
    ylabel('Latitud')
    title(['Divergencia del viento el ' dia, '-', mes, '-', year, ' a las ', hora, ':', mins ' GMT, t=' num2str(tt)], 'FontSize', 13)
    
    pause(0.1)
    %     if tt==27
    %         name=(['rot_enr_0_t' num2str(tt) '_OK' ]);
    %         print('-djpeg', name)
    %     end
end



 Ejercicio libro capítulo finite differences
    
    Determine the order of accuracy of the centred discretisations of the advection schemeun+1j−un−1j2∆t+cunj+1−unj−12∆x= 0.